In [1]:
import numpy as np 
import pandas as pd 
from langdetect import detect
import time
import pickle

# IMPORT

In [2]:
df_info = pd.read_csv('raw_data/metacritic_game_info.csv')
df_info.head(3)

,Unnamed: 0,Title,Year,Publisher,Genre,Platform,Metascore,Avg_Userscore,No_Players
0,0,The Legend of Zelda: Ocarina of Time,1998,Nintendo,Action Adventure;Fantasy,Nintendo64,99,9.1,1 Player
1,1,Tony Hawk's Pro Skater 2,2000,NeversoftEntertainment,Sports;Alternative;Skateboarding,PlayStation,98,7.4,1-2
2,2,Grand Theft Auto IV,2008,RockstarNorth,Action Adventure;Modern;Modern;Open-World,PlayStation3,98,7.5,1 Player


In [3]:
df_comments = pd.read_csv('raw_data/metacritic_game_user_comments.csv')
df_comments.head(3)

,Unnamed: 0,Title,Platform,Userscore,Comment,Username
0,0,The Legend of Zelda: Ocarina of Time,Nintendo64,10,"Everything in OoT is so near at perfection, it...",SirCaestus
1,1,The Legend of Zelda: Ocarina of Time,Nintendo64,10,I won't bore you with what everyone is already...,Kaistlin
2,2,The Legend of Zelda: Ocarina of Time,Nintendo64,10,Anyone who gives the masterpiece below a 7 or ...,Jacody


In [4]:
df_info.shape, df_comments.shape

((5000, 9), (283983, 6))

In [5]:
df_comments.Userscore.value_counts(normalize=True)

10    0.396260
9     0.188353
8     0.106594
0     0.070849
7     0.060046
6     0.042383
5     0.035611
4     0.028002
1     0.025769
3     0.025065
2     0.021068
Name: Userscore, dtype: float64

# Metacritic Game Info

In [6]:
# remove the extra index

df_info = df_info.drop(columns='Unnamed: 0')
df_info.head(1)

,Title,Year,Publisher,Genre,Platform,Metascore,Avg_Userscore,No_Players
0,The Legend of Zelda: Ocarina of Time,1998,Nintendo,Action Adventure;Fantasy,Nintendo64,99,9.1,1 Player


In [7]:
df_info.nunique()

Title            3438
Year               25
Publisher        1238
Genre             927
Platform           20
Metascore          24
Avg_Userscore      70
No_Players         52
dtype: int64

In [9]:
# remove duplicates

print(f'{df_info.duplicated().sum()} duplicates have been removed.')

df_info.drop_duplicates(inplace=True)

7 duplicates have been removed.


- year

In [10]:
# remove 'not specified' in column Year
# 4 of them

t = df_info[df_info['Year'] == 'not specified'].shape[0]
print(f'{t} nans have been removed.')

drop_index = df_info[df_info['Year'] == 'not specified'].index
df_info.drop(drop_index, inplace=True)

4 nans have been removed.


- No_Players

In [16]:
df_info['No_Players'].value_counts()

not specified            1539
No Online Multiplayer     852
1 Player                  573
1-2                       351
1-4                       334
1-8                       127
2  Online                 123
4  Online                 113
Up to 4                   103
8  Online                  94
Online Multiplayer         81
Up to 8                    78
2                          74
Massively Multiplayer      52
16  Online                 49
1-16                       47
Up to 10                   32
Up to 16                   32
6  Online                  32
Up to 6                    30
Up to 12                   28
32  Online                 26
Up to 18                   24
10  Online                 20
1-32                       18
Up to 22                   17
1-6                        16
12  Online                 15
Up to 24                   10
1-3                         9
Up to 20                    9
24  Online                  8
1-10                        7
Up to 64  

In [15]:
df_info[df_info['No_Players'].isna() == True] = 'not specified'

In [17]:
df_info.columns

Index(['Title', 'Year', 'Publisher', 'Genre', 'Platform', 'Metascore',
       'Avg_Userscore', 'No_Players'],
      dtype='object')

In [18]:
# a quick export for easy remsy

df_info.to_csv(r'processed_remsy\metacritic_info_remsy.csv', index = False)

# Metacritic Review

In [11]:
# remove the extra index

df_comments = df_comments.drop(columns='Unnamed: 0')
df_comments.head(1)

,Title,Platform,Userscore,Comment,Username
0,The Legend of Zelda: Ocarina of Time,Nintendo64,10,"Everything in OoT is so near at perfection, it...",SirCaestus


In [12]:
df_comments.nunique()

Title          2325
Platform         20
Userscore        11
Comment      277706
Username     133664
dtype: int64

In [4]:
df_comments.shape

(283983, 5)

In [5]:
df_comments.drop_duplicates(inplace=True)
df_comments.dropna(inplace=True)

In [6]:
df_comments.shape

(282201, 5)

# langauage detect

In [58]:
df_comments_5000 = df_comments.sample(5000)

In [66]:
def detect_lang(t):
    try:
        lang = detect(t)
    except:
        lang = 'error'

    return lang


In [69]:
start = time.time()

df_comments_5000['lang'] = df_comments_5000['Comment'].apply(detect_lang)

end = time.time()
print(f'this function took {end-start} seconds.')

this function took 23.219751358032227 seconds.


In [71]:
df_comments_5000

,Title,Platform,Userscore,Comment,Username,lang
214052,Colin McRae Rally 04,Xbox,7,Certainly worth $20 ... but not much more.,Wes,en
187241,Dragon Age: Inquisition,PC,10,All these negative reviews are fake or from bo...,Pecheckler,en
96156,Dota 2,PC,0,Nothing but hats aka. cosmetics for the longes...,edmundduke,en
270806,Final Fantasy XV,PlayStation4,5,I literally grew up playing Final Fantasy game...,Tigerstrike11,en
167565,Gone Home,PC,1,Over rated game that has a surprisingly boring...,Orionsbelt,en
...,...,...,...,...,...,...
140367,Diablo III,PC,0,It doesn't much matter what's in the game if t...,ImmortalTrout,en
207670,Middle-earth: Shadow of Mordor,PC,9,Quick version:It's like an Assassins Creed tha...,Alsmir,en
104300,Dark Souls III,PC,9,Although there has been a lot of bugs being re...,MarcioCavalcant,en
45032,The Witcher 3: Wild Hunt,PC,10,Perfect Game.Probably best game ever.Graphics:...,roi123,en


In [74]:
df_comments_5000[df_comments_5000.lang=='error']

,Title,Platform,Userscore,Comment,Username,lang
32780,Batman: Arkham City,Xbox360,7,Ð,Jivot,error


In [30]:
df_comments_5000['lang'].value_counts(normalize=True)

en    0.6050
de    0.0616
es    0.0330
id    0.0308
tl    0.0298
fr    0.0220
it    0.0218
nl    0.0204
cy    0.0194
af    0.0154
so    0.0152
no    0.0144
da    0.0132
pt    0.0132
ca    0.0126
fi    0.0114
sw    0.0104
hu    0.0080
pl    0.0080
ro    0.0068
sl    0.0042
sq    0.0040
tr    0.0034
sv    0.0032
et    0.0032
hr    0.0030
sk    0.0028
vi    0.0018
lt    0.0010
lv    0.0006
cs    0.0004
Name: lang, dtype: float64

In [75]:
start = time.time()

df_comments['lang'] = df_comments['Comment'].apply(detect_lang)

end = time.time()
print(f'this function took {end-start} seconds.')

this function took 1269.3497936725616 seconds.


pickle

In [88]:
df_comments.to_pickle('processed_df/metacritic_comment_langdetected.pkl')

In [7]:
df_comments = pickle.load(open('processed_df/metacritic_comment_langdetected.pkl','rb'))

In [8]:
df_comments

,Title,Platform,Userscore,Comment,Username,lang
0,The Legend of Zelda: Ocarina of Time,Nintendo64,10,"Everything in OoT is so near at perfection, it...",SirCaestus,en
1,The Legend of Zelda: Ocarina of Time,Nintendo64,10,I won't bore you with what everyone is already...,Kaistlin,en
2,The Legend of Zelda: Ocarina of Time,Nintendo64,10,Anyone who gives the masterpiece below a 7 or ...,Jacody,en
3,The Legend of Zelda: Ocarina of Time,Nintendo64,10,I'm one of those people who think that this is...,doodlerman,en
4,The Legend of Zelda: Ocarina of Time,Nintendo64,10,This game is the highest rated game on Metacr...,StevenA,en
...,...,...,...,...,...,...
283978,Etrian Odyssey Untold: The Millennium Girl,3DS,7,"Extremely similar to EO:4, which obviously isn...",RileyWRussell,en
283979,Etrian Odyssey Untold: The Millennium Girl,3DS,0,Typical overrated Atlus trash. A game i should...,TemplarGR,en
283980,Etrian Odyssey Untold: The Millennium Girl,3DS,9,While I find the story mode to have annoying c...,midipon,en
283981,Etrian Odyssey Untold: The Millennium Girl,3DS,8,"Pretty good, but it certainly lacks the visual...",night4,en


In [9]:
df_comments['lang'].value_counts(normalize=True)

en       0.956648
es       0.016595
pt       0.006683
ru       0.005464
fr       0.002658
de       0.001956
it       0.001325
da       0.000797
af       0.000794
so       0.000769
tr       0.000734
pl       0.000560
no       0.000489
tl       0.000415
nl       0.000393
error    0.000326
ca       0.000322
cs       0.000305
ro       0.000262
ja       0.000259
hu       0.000259
cy       0.000245
et       0.000195
id       0.000188
ko       0.000174
fi       0.000159
sl       0.000145
vi       0.000117
hr       0.000106
zh-cn    0.000106
sv       0.000096
sw       0.000085
sq       0.000082
fa       0.000057
lt       0.000053
sk       0.000043
lv       0.000028
el       0.000025
uk       0.000021
ar       0.000014
bg       0.000011
th       0.000011
mk       0.000011
he       0.000007
zh-tw    0.000007
ta       0.000004
Name: lang, dtype: float64

# EXPORT

In [87]:
df_comments.columns

Index(['Title', 'Platform', 'Userscore', 'Comment', 'Username', 'lang'], dtype='object')

In [86]:
df_comments.shape

(282201, 6)

- NLP

In [89]:
col_sel = ['Title', 'Userscore', 'Comment', 'lang']
df_comments_nlp = df_comments[col_sel]

In [90]:
print(f'shape before: {df_comments_nlp.shape}')
df_comments_nlp = df_comments_nlp[df_comments_nlp['lang'] == 'en']
print(f'shape after removed non-english: {df_comments_nlp.shape}')

shape before: (282201, 4)
shape after removed non-english: (269967, 4)


In [91]:
# replace userscore 0.0 with nan
# assume 0.0 means not rated
# this df is for NLP EDA and modeling, doesn't need missing values

df_comments_nlp.loc[df_comments_nlp['Userscore'] == 0, 'Userscore'] = np.NAN


In [92]:
print(f'shape before: {df_comments_nlp.shape}')

df_comments_nlp.drop_duplicates(inplace=True)
df_comments_nlp.dropna(inplace=True)

print(f'shape after removed score 0.0: {df_comments_nlp.shape}')

shape before: (269967, 4)
shape after removed score 0.0: (249031, 4)


In [93]:
df_comments_nlp.to_csv(r'processed_df\metacritic_comments_nlp.csv', index = False)